In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
import pickle

In [8]:
#Getting the Data
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
#drop useless columns
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [10]:
encoder=LabelEncoder()
data['Gender']=encoder.fit_transform(data['Gender'])
data


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [11]:
#OneHot encoding for geography
from sklearn.preprocessing import OneHotEncoder
Encode_geo=OneHotEncoder()
new_geo=Encode_geo.fit_transform(data[['Geography']])
new_geo

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [12]:
#Columns of new Geography
Encode_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [13]:
#Converting the new Geography into data frame (new_geo is in sparse format we should convert it into array)
Encoded_feature=pd.DataFrame(new_geo.toarray(),columns=Encode_geo.get_feature_names_out(['Geography']))

In [14]:
Encoded_feature

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [15]:
#Drop the Geography column and add the Encoded_feature
data=pd.concat([data.drop('Geography',axis=1),Encoded_feature],axis=1)

In [16]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [17]:
#save the one hot encoder and label encoder in pickle
with open('label_encoder.pkl','wb') as file:
    pickle.dump(encoder,file)

with open('one_hot_encoder.pkl','wb') as file:
    pickle.dump(Encode_geo,file)     

In [18]:
# Divide data into Dependent and Independent Features
X=data.drop('Exited',axis=1)
y=data['Exited']

In [19]:
#Do train Test Split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42 )

In [20]:
# Use standard Scaler
scaler=StandardScaler()
# Step 1: Fit only on training data
#scaler.fit(X_train)

# Step 2: Transform training and test data
#X_train_scaled = scaler.transform(X_train)
#X_test_scaled = scaler.transform(X_test)

X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [21]:
#Store it in pickle file
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

                                       ANN IMPLEMANTATION 

In [22]:
import tensorflow as tf

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [29]:
#Build Model


In [31]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1 connected to input layer
    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid') 
])

In [32]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:

opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()

In [36]:
#Compile the model
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [ ]:
log_dir='logs/fit/'

In [39]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [40]:
#setUp Early Stopping
early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [41]:
#Training the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8363 - loss: 0.4012 - val_accuracy: 0.8530 - val_loss: 0.3544
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8541 - loss: 0.3539 - val_accuracy: 0.8600 - val_loss: 0.3584
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8546 - loss: 0.3501 - val_accuracy: 0.8590 - val_loss: 0.3497
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8593 - loss: 0.3432 - val_accuracy: 0.8545 - val_loss: 0.3610
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8583 - loss: 0.3449 - val_accuracy: 0.8525 - val_loss: 0.3522
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8591 - loss: 0.3373 - val_accuracy: 0.8620 - val_loss: 0.3344
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8622 - loss: 0.3383 - val_accuracy: 0.8535 - val_loss: 0.3408
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8614 - loss: 0.3330 - val_accu

In [42]:
model.save('model.h5')

In [43]:
%load_ext tensorboard

In [45]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 4368), started 0:07:35 ago. (Use '!kill 4368' to kill it.)